In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import math
import random

import numpy as np
import pytest
import torch
import scipy.optimize
import socialforce


# OPTIMIZER_OPT = {'eps': 1e-4, 'gtol': 1e-4, 'maxcor': 30, 'maxls': 10, 'disp': True}
OPTIMIZER_OPT = {'eps': 0.01, 'disp': True}

[Sven Kreiss](https://www.svenkreiss.com), April 17 2020

# Fit PedPed Potential

In [ ]:
def visualize(file_prefix, V, initial_state_dict, final_state_dict, fit_result=None, V_gen=None):
    b = np.linspace(0, 3, 200)
    y_ref = 2.1 * np.exp(-1.0 * b / 0.3)
    if V_gen is not None:
        y_ref = V_gen.v0 * np.exp(-1.0 * b / V_gen.sigma)

    V.load_state_dict(initial_state_dict)
    y_initial = V.value_b(torch.from_numpy(b)).detach().numpy()
    y_initial -= y_initial[-1]

    V.load_state_dict(final_state_dict)
    y_mlp = V.value_b(torch.from_numpy(b)).detach().numpy()
    y_mlp -= y_mlp[-1]

    with socialforce.show.canvas(file_prefix + 'v_gradv.png', ncols=2) as (ax1, ax2):
        ax1.set_xlabel('$b$ [m]')
        ax1.set_ylabel('$V$')
        ax1.plot(b, y_ref, label=r'true $V_0 e^{-b/\sigma}$', color='gray')
        ax1.axvline(0.3, color='gray', linestyle='dotted', label=r'true $\sigma$')
        ax1.plot(b, y_initial, label=r'initial MLP($b$)',
                linestyle='dashed', color='C0')
        ax1.plot(b, y_mlp, label=r'MLP($b$)', color='C0')
        ax1.legend()

        ax2.set_xlabel(r'$b$ [m]')
        ax2.set_ylabel(r'$\nabla V$')
        delta_b = b[1:] - b[:-1]
        average_b = 0.5 * (b[:-1] + b[1:])
        ax2.plot(average_b, (y_ref[1:] - y_ref[:-1]) / delta_b,
                label=r'true $V_0 e^{-b/\sigma}$', color='gray')
        ax2.axvline(0.3, color='gray', linestyle='dotted', label=r'true $\sigma$')
        ax2.plot(average_b, (y_initial[1:] - y_initial[:-1]) / delta_b,
                label=r'initial MLP($b$)',
                linestyle='dashed', color='C0')
        ax2.plot(average_b, (y_mlp[1:] - y_mlp[:-1]) / delta_b,
                label=r'MLP($b$)', color='C0')
        ax2.set_ylim(-4.9, 0.5)
        ax2.legend()

## Generate Truth for Opposing Scenario

Left: $\Delta t = 0.4s$  
Right: $\Delta t = 0.05s$

In [ ]:
initial_state = [
    [0.0, 0.0, 0.0, 1.0, 0.0, 10.0],
    [-0.3, 10.0, 0.0, -1.0, -0.3, 0.0],
]

truth_nooversampling = socialforce.Simulator(initial_state, oversampling=1).run(21).detach()
truth = socialforce.Simulator(initial_state).run(21).detach()

with socialforce.show.track_canvas(ncols=2) as (ax1, ax2):
    socialforce.show.states(ax1, truth_nooversampling)
    socialforce.show.states(ax2, truth)

## Fit a Potential with Manual Gradient Descent Implementation

In [ ]:
# v0 = torch.tensor(1.2, dtype=torch.double, requires_grad=True)
# sigma_v = torch.tensor(0.1, dtype=torch.double, requires_grad=True)
# V = socialforce.PedPedPotential(v0, sigma_v)
# initial_states = socialforce.Simulator(initial_state, ped_ped=V, oversampling=1).run(21)

# for i_update in range(100):
#     generated_states = socialforce.Simulator(initial_state, ped_ped=V, oversampling=1).run(21, detach=False)
#     # generated_states = socialforce.Simulator(initial_state, ped_ped=V, oversampling=1, delta_t=0.04).run(210, detach=False)
#     loss_l2 = (generated_states[1:, :, :2]-truth_nooversampling[1:, :, :2]).norm(dim=-1).sum()
#     # loss_l2 = (generated_states[10::10, :, :2]-truth[1:, :, :2]).norm(dim=-1).sum()
#     loss_constantv = (generated_states[2:, :, 2:4] - generated_states[1:-1, :, 2:4]).norm(dim=-1).sum()
#     loss = loss_l2 + 0.0 * loss_constantv
#     print('losses', loss, loss_l2, loss_constantv)

#     v0_grad, sigma_grad = torch.autograd.grad(loss, [v0, sigma_v])

#     lr = 0.05
#     with torch.no_grad():
#         v0 -= lr * v0_grad
#         sigma_v -= lr * sigma_grad

#     print('v0', v0, v0_grad)
#     print('sigma', sigma_v, sigma_grad)

# assert v0.item() == pytest.approx(2.1, abs=0.01)
# assert sigma_v.item() == pytest.approx(0.3, abs=0.01)

In [ ]:
v0 = torch.tensor(1.2, dtype=torch.double, requires_grad=True)
sigma_v = torch.tensor(0.1, dtype=torch.double, requires_grad=True)
V = socialforce.PedPedPotential(v0, sigma_v)
initial_states = socialforce.Simulator(initial_state, ped_ped=V).run(21).detach()

for i_update in range(100):
    generated_states = socialforce.Simulator(initial_state, ped_ped=V).run(21)
    # generated_states = socialforce.Simulator(initial_state, ped_ped=V, oversampling=1, delta_t=0.04).run(210, detach=False)
    loss = (generated_states[1:, :, :2]-truth[1:, :, :2]).norm(dim=-1).sum()

    v0_grad, sigma_grad = torch.autograd.grad(loss, [v0, sigma_v])

    lr = 0.3 
    if i_update > 50:
        lr = 0.03
    elif i_update > 30:
        lr = 0.1
    with torch.no_grad():
        v0 -= lr * v0_grad
        sigma_v -= lr * sigma_grad

    print(i_update, 'v0', lr, v0.item())
    # print('sigma', sigma_v, sigma_grad)

assert v0.item() == pytest.approx(2.1, abs=0.05)
assert sigma_v.item() == pytest.approx(0.3, abs=0.03)

In [ ]:
v0 = torch.tensor(1.2, dtype=torch.double, requires_grad=True)
sigma_v = torch.tensor(0.1, dtype=torch.double, requires_grad=True)
V = socialforce.PedPedPotential(v0, sigma_v)
initial_states = socialforce.Simulator(initial_state, ped_ped=V).run(21).detach()

for i_update in range(100):
    generated_states = socialforce.Simulator(initial_state, ped_ped=V).run(21)
    # generated_states = socialforce.Simulator(initial_state, ped_ped=V, oversampling=1, delta_t=0.04).run(210, detach=False)
    losses = (generated_states[1:, :, :2]-truth[1:, :, :2]).norm(dim=-1)

    # curriculum: shorter paths at the beginning
    if i_update < 50:
        losses = losses[:15]

    loss = losses.sum()

    v0_grad, sigma_grad = torch.autograd.grad(loss, [v0, sigma_v])

    lr = 0.3 
    if i_update > 50:
        lr = 0.03
    elif i_update > 30:
        lr = 0.1
    with torch.no_grad():
        v0 -= lr * v0_grad
        sigma_v -= lr * sigma_grad

    print(i_update, 'v0', lr, v0.item())
    # print('sigma', sigma_v, sigma_grad)

assert v0.item() == pytest.approx(2.1, abs=0.05)
assert sigma_v.item() == pytest.approx(0.3, abs=0.03)

In [ ]:
final_states = socialforce.Simulator(initial_state, ped_ped=V).run(21).detach()
with socialforce.show.track_canvas(ncols=2) as (ax1, ax2):
    socialforce.show.states(ax1, truth, color='grey')
    socialforce.show.states(ax1, initial_states)
    socialforce.show.states(ax2, truth, color='grey')
    socialforce.show.states(ax2, final_states)

## Fit a Potential with Numerical Gradient Approximation with SciPy

In [ ]:
v0 = torch.tensor(1.2, dtype=torch.double, requires_grad=True)
sigma_v = torch.tensor(0.1, dtype=torch.double, requires_grad=True)
V = socialforce.PedPedPotential(v0, sigma_v)
initial_states = socialforce.Simulator(initial_state, ped_ped=V).run(21).detach()

In [ ]:
def f(x):
    V = socialforce.PedPedPotential(float(x[0]), float(x[1]))
    with torch.no_grad():
        return socialforce.Simulator(initial_state, ped_ped=V).run(21)

def loss(x):
    generated_states = f(x)
    return (generated_states[1:, :, :2] - truth[1:, :, :2]).norm(dim=-1).sum()

parameters = np.array([1.2, 0.1])
res = scipy.optimize.minimize(loss, parameters, method='L-BFGS-B', options=OPTIMIZER_OPT)
print(res)

# assert res.x == pytest.approx(np.array([2.1, 0.3]), abs=0.01)
assert res.fun < 0.05 * 20  # less than 5cm error on average over 20 steps

In [ ]:
final_states = f(res.x)
with socialforce.show.track_canvas(ncols=2) as (ax1, ax2):
    socialforce.show.states(ax1, truth, color='grey')
    socialforce.show.states(ax1, initial_states)
    socialforce.show.states(ax2, truth, color='grey')
    socialforce.show.states(ax2, final_states)

# Fit an MLP Potential with Numerical Gradient with SciPy

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

V = socialforce.PedPedPotentialMLP().double()
parameters = V.get_parameters().clone().detach().numpy()
initial_parameters = parameters.copy()

# training
def f(x):
    V.set_parameters(torch.from_numpy(x))
    with torch.no_grad():
        generated_states = socialforce.Simulator(initial_state, ped_ped=V).run(21)

    loss = (generated_states[1:, :, :2] - truth[1:, :, :2]).norm(dim=-1).sum()

    return loss

res = scipy.optimize.minimize(f, parameters, method='L-BFGS-B', options=OPTIMIZER_OPT)
print(res)

In [ ]:
visualize('images/mlp_scipy_', V, initial_parameters, res.x)

## Fitting to 1, 5 and 20 Circle Scenarios each with Two Pedestrians

The preferred speed needs to be varied. Otherwise the symmetry of the problem creates unrealistic scenarios where the two pedestrians get stuck.

In [ ]:
def generate_gt(n):
    torch.manual_seed(42)
    np.random.seed(42)

    # ped0 always left to right
    ped0 = np.array([-5.0, 0.0, 1.0, 0.0, 5.0, 0.0])

    generator_initial_states = []
    for theta in np.random.rand(n) * 2.0 * math.pi:
        # ped1 at a random angle with +/-20% speed variation
        c, s = np.cos(theta), np.sin(theta)
        r = np.array([[c, -s], [s, c]])
        ped1 = np.concatenate((
            np.matmul(r, ped0[0:2]),
            np.matmul(r, ped0[2:4]) * (0.8 + np.random.rand(1) * 0.4),
            np.matmul(r, ped0[4:6]),
        ))
        generator_initial_states.append(
            np.stack((ped0, ped1))
        )

    generator_ped_ped = socialforce.PedPedPotential(2.1)
    with torch.no_grad():
        trajectories = [
            socialforce.Simulator(initial_state, ped_ped=generator_ped_ped).run(21)
            for initial_state in generator_initial_states
        ]
    return generator_ped_ped, trajectories

generator_pedped_1, scenarios_1 = generate_gt(1)
generator_pedped_5, scenarios_5 = generate_gt(5)
generator_pedped_20, scenarios_20 = generate_gt(20)

In [ ]:
with socialforce.show.track_canvas(ncols=3, figsize=(12, 4)) as (ax1, ax2, ax3):
    socialforce.show.states(ax1, scenarios_1[0])

    socialforce.show.states(ax2, scenarios_5[-1])
    for scene in scenarios_5[:-1]:
        socialforce.show.states(ax2, scene, alpha=0.1)

    socialforce.show.states(ax3, scenarios_20[-1])
    for scene in scenarios_20[:-1]:
        socialforce.show.states(ax3, scene, alpha=0.1)

In [ ]:
true_experience = socialforce.Trainer.scenes_to_experience(scenarios_1)
V = socialforce.PedPedPotentialMLP().double()
initial_parameters = V.state_dict()

def simulator_factory(initial_state):
    return socialforce.Simulator(initial_state, ped_ped=V)

opt = torch.optim.SGD(V.parameters(), lr=10.0)
socialforce.Trainer(simulator_factory, opt, true_experience, batch_size=1).loop(50)

In [ ]:
# make plots of result
visualize('images/mlp_circle_n{}_'.format(1), V, initial_parameters, V.state_dict(), V_gen=generator_pedped_1)

In [ ]:
true_experience = socialforce.Trainer.scenes_to_experience(scenarios_5)
V = socialforce.PedPedPotentialMLP().double()
initial_parameters = V.state_dict()

def simulator_factory(initial_state):
    return socialforce.Simulator(initial_state, ped_ped=V)

opt = torch.optim.SGD(V.parameters(), lr=1.0)
socialforce.Trainer(simulator_factory, opt, true_experience, batch_size=1).loop(100)

In [ ]:
visualize('images/mlp_circle_n{}_'.format(5), V, initial_parameters, V.state_dict(), V_gen=generator_pedped_5)